In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import FreqDist


import re
import string
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from gensim.models.doc2vec import Doc2Vec, TaggedDocument


import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import *
from keras.models import *
from keras.layers import *

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from wordcloud import WordCloud, STOPWORDS 



import nltk
import csv
import sys
import pandas as pd

In [2]:
import feedparser

url = "http://rss.cnn.com/rss/cnn_topstories.rss"
feed = feedparser.parse(url)

print(feed)

{'bozo': False, 'entries': [{'title': 'Some on-air claims about Dominion Voting Systems were false, Fox News acknowledges in statement after deal is announced', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'http://rss.cnn.com/rss/cnn_topstories.rss', 'value': 'Some on-air claims about Dominion Voting Systems were false, Fox News acknowledges in statement after deal is announced'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.cnn.com/business/live-news/fox-news-dominion-trial-04-18-23/index.html'}], 'link': 'https://www.cnn.com/business/live-news/fox-news-dominion-trial-04-18-23/index.html', 'id': 'https://www.cnn.com/business/live-news/fox-news-dominion-trial-04-18-23/index.html', 'guidislink': False, 'published': 'Wed, 19 Apr 2023 12:44:51 GMT', 'published_parsed': time.struct_time(tm_year=2023, tm_mon=4, tm_mday=19, tm_hour=12, tm_min=44, tm_sec=51, tm_wday=2, tm_yday=109, tm_isdst=0), 'media_content': [{'medium': 'image', 'url': 'https

In [3]:
print("Feed Title:", feed.feed.title)
print("Feed Description:", feed.feed.description)
print("Feed Link:", feed.feed.link)
print("Feed Link:", feed.feed.language)

Feed Title: CNN.com - RSS Channel - HP Hero
Feed Description: CNN.com delivers up-to-the-minute news and information on the latest top stories, weather, entertainment, politics and more.
Feed Link: https://www.cnn.com/index.html
Feed Link: en-US


In [4]:
for entry in feed.entries:
    print("Entry Title:", entry.title)
    print("Entry Link:", entry.link)
    print("\n")

Entry Title: Some on-air claims about Dominion Voting Systems were false, Fox News acknowledges in statement after deal is announced
Entry Link: https://www.cnn.com/business/live-news/fox-news-dominion-trial-04-18-23/index.html


Entry Title: Dominion still has pending lawsuits against election deniers such as Rudy Giuliani and Sidney Powell
Entry Link: https://www.cnn.com/business/live-news/fox-news-dominion-trial-04-18-23/h_8d51e3ae2714edaa0dace837305d03b8


Entry Title: Here are the 20 specific Fox broadcasts and tweets Dominion says were defamatory
Entry Link: https://www.cnn.com/2023/04/17/media/dominion-fox-news-allegations/index.html


Entry Title: Judge in Fox News-Dominion defamation trial: 'The parties have resolved their case'
Entry Link: https://www.cnn.com/2023/04/18/media/fox-dominion-settlement/index.html


Entry Title: 'Difficult to say with a straight face': Tapper reacts to Fox News' statement on settlement
Entry Link: https://www.cnn.com/videos/politics/2023/04/18/ja

In [5]:
import csv

# Prepare the CSV file
with open('rss_data.csv', mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['title', 'link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    # Iterate through entries and write to the CSV file
    for entry in feed.entries:
        writer.writerow({'title': entry.title, 'link': entry.link,})

In [6]:
data=[]
def fetch_rss_data(url):
    feed = feedparser.parse(url)
    print("Feed Title:", feed.feed.title)
    for entry in feed.entries:
        data.append([entry.title,entry.link])
        print("Entry Title:", entry.title)
        print("Entry Link:", entry.link)
        try:
            if 'tags' in entry:
                categories = [tag.term for tag in entry.tags]
                print("Categories:", ", ".join(categories))
        except:
            print("Category : not their")
        try:
            print("Entry Description:", remove_punctuaction(entry.description))
        except:
            print("Description : not their")
        print("\n")
        
# List of RSS feed URLs
rss_feed_urls = [
    
    #"http://qz.com/feed",#<div class="sc-11qwj9y-0 gkxmUx">
    "http://feeds.foxnews.com/foxnews/politics",#<div class="article-body">
    #"http://feeds.feedburner.com/NewshourWorld",#<div class="post__col-right">
    #"http://rss.cnn.com/rss/cnn_topstories.rss",#<div class="article__content" data-editable="content" itemprop="articleBody" data-reorderable="content">
    #"https://feeds.bbci.co.uk/news/world/asia/india/rss.xml"#<article class="ssrcss-pv1rh6-ArticleWrapper e1nh2i2l5",,,<div class="ssrcss-1ki8hfp-StyledZone e1mcntqj3">
    
]
# Fetch data from multiple RSS feeds


for url in rss_feed_urls:
    fetch_rss_data(url)


Feed Title: Latest Political News on Fox News
Entry Title: Michael Cohen to testify in Trump trial on Monday
Entry Link: https://www.foxnews.com/politics/michael-cohen-testify-trump-trial-monday
Categories: 76556ed8-dad4-502b-b1d2-b5a2ed829422, fnc, Fox News, fox-news/politics, article
Description : not their


Entry Title: Blue state Dems turn on governor as homeless council can't account for $20B in spending
Entry Link: https://www.foxnews.com/politics/blue-state-dems-turn-governor-homeless-council-cant-account-20b-spending
Categories: 4e7200dc-3999-568f-9924-ea9cf7c25fe8, fnc, Fox News, fox-news/newsedge/politics, fox-news/us/us-regions/west/california, fox-news/person/gavin-newsom, fox-news/topic/homeless-crisis, fox-news/politics, article
Description : not their


Entry Title: Shots fired at Border Patrol agents investigated as ‘assault on a federal officer’: FBI
Entry Link: https://www.foxnews.com/politics/shots-fired-border-patrol-agents-investigated-assault-federal-officer-fbi


In [31]:
from bs4 import BeautifulSoup
import requests
import re
datas = []
def extract(url):
    response=requests.get(url)
    if response.status_code==200:
        soup=BeautifulSoup(response.text,'html.parser')
        headlines=soup.find_all('div',class_="article-body" )
        if headlines:
            for headline in headlines:
                extracted_text = headline.text.strip( )  # Clean whitespace
                # Process or store the extracted text as needed
                print(extracted_text)
                dfs_clean=message_cleaning(extracted_text)
                dfs_cleaner = ["".join(x) for x in dfs_clean]
                print(len(dfs_cleaner))
                datas.append(dfs_cleaner)
                print("\n")
        else:
            print("error")
    else:
        print(f'Error{response.status_code}')



for e in data:
    print(e[1])
    extract(e[1])
                



https://www.foxnews.com/politics/michael-cohen-testify-trump-trial-monday
close      Video Michael Cohen set to testify in NY v Trump trial in Manhattan  Fox News legal editor Kerri Urbahn joined Fox & Friends to discuss her take on what Michael Cohens testimony will do to the case and why she believes Stormy Daniels details were inconsistent.  Michael Cohen, ex-lawyer for former President Donald Trump, is expected to testify on Monday, Fox News has learned.     Cohen is expected to be the star witness for Manhattan District Attorney Alvin Bragg and his team for his role in arranging a $130,000 payment in 2016 to adult actress Stormy Daniels to reportedly keep her story that she had a sexual encounter with Trump in the early 2000s. Trump, later, made several payments of $35,000 to Cohen, who was his personal attorney at the time. Trump has denied the encounter happened.  Those payments served as the basis for Bragg's indictment of Trump on charges of falsifying business records. Bragg 

In [32]:
dfs = pd.DataFrame(datas)
print(dfs.shape)
combined_data = []

for index, row in dfs.iterrows():
    combined_text = " ".join([x for x in row.tolist() if x is not None])
    combined_data.append(combined_text)
                    
df_combined = pd.DataFrame(combined_data, columns=['combined_column'])
dfs_combined=df_combined.squeeze()
print(dfs_combined.head())



(25, 852)
0    close Video Michael Cohen set testify NY v Tru...
1    close Video Newsom California homelessness cri...
2    close Video New Mexico border area becoming fr...
3    close Video Fox News Flash top headlines May 9...
4    close Video Rep Issa reacts growing antisemiti...
Name: combined_column, dtype: object


In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
df = pd.read_csv(r"BBC News Train.csv", encoding= 'ISO-8859-1')

In [11]:
df=df.drop(["Unnamed: 3"],axis=1)
df=df.drop(["ArticleId"],axis=1)
print(df.shape)
print(df.head())
print(" ")
df.info()


(1490, 2)
                                                Text       Category
0  worldcom boss  left books alone  former worldc...       business
1  yeading face newcastle in fa cup premiership s...          sport
2  blair prepares to name poll date tony blair is...       politics
3  last star wars  not for children  the sixth an...  entertainment
4  berlin cheers for anti-nazi film a german movi...  entertainment
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      1490 non-null   object
 1   Category  1490 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB


In [12]:
from nltk.corpus import stopwords
import string

stopwords_list = stopwords.words('english')

# add customized stopwords from my dataset
# additional_words = ['“','”','...','``',"''",'’',"httpst",'amp',"theto","}",
#                     "{","rt",".codgy3azxwOY","http:","https:","RT" ,"rt"]

stopwords_list+=string.punctuation

In [13]:
def message_cleaning(message):
  ''' pipeline function performs the follwoing:
  (1) remove punctiation,
  (2) remove stopwords'''


  Test_punc_removed=[char for char in message if char not in string.punctuation]
  Test_punc_removed_join=''.join(Test_punc_removed)
  Test_punc_removed_join_clean=[word for word in Test_punc_removed_join.split() if word.lower() not in stopwords_list ]
  return Test_punc_removed_join_clean
    

In [14]:
df_clean=df['Text'].apply(message_cleaning)
df_clean.head()
print(df_clean.head())

0    [worldcom, boss, left, books, alone, former, w...
1    [yeading, face, newcastle, fa, cup, premiershi...
2    [blair, prepares, name, poll, date, tony, blai...
3    [last, star, wars, children, sixth, final, sta...
4    [berlin, cheers, antinazi, film, german, movie...
Name: Text, dtype: object

0    [worldcom, boss, left, books, alone, former, w...
1    [yeading, face, newcastle, fa, cup, premiershi...
2    [blair, prepares, name, poll, date, tony, blai...
3    [last, star, wars, children, sixth, final, sta...
4    [berlin, cheers, antinazi, film, german, movie...
Name: Text, dtype: object


In [15]:
df_cleaner = [" ".join(x) for x in df_clean]

In [16]:
new_df = pd.DataFrame(df_cleaner, columns=["Text"])
new_df.head()

,Text
0,worldcom boss left books alone former worldcom...
1,yeading face newcastle fa cup premiership side...
2,blair prepares name poll date tony blair likel...
3,last star wars children sixth final star wars ...
4,berlin cheers antinazi film german movie antin...


In [17]:
df["Category"].value_counts()

Category
sport            346
business         336
politics         274
entertainment    273
tech             261
Name: count, dtype: int64

In [18]:
df["Categoryid"] = df["Category"].factorize()[0]
df.head()

,Text,Category,Categoryid
0,worldcom boss left books alone former worldc...,business,0
1,yeading face newcastle in fa cup premiership s...,sport,1
2,blair prepares to name poll date tony blair is...,politics,2
3,last star wars not for children the sixth an...,entertainment,3
4,berlin cheers for anti-nazi film a german movi...,entertainment,3


In [19]:
category = df[["Category","Categoryid"]]

In [20]:
new_df['length']=new_df['Text'].apply(len)
new_df['length']

0       1380
1       1809
2       1021
3        843
4       1328
        ... 
1485     715
1486    1379
1487    1274
1488    1147
1489    2819
Name: length, Length: 1490, dtype: int64

In [21]:
new_df["Category"]=df["Category"]

In [22]:
print(new_df)

                                                   Text  length       Category
0     worldcom boss left books alone former worldcom...    1380       business
1     yeading face newcastle fa cup premiership side...    1809          sport
2     blair prepares name poll date tony blair likel...    1021       politics
3     last star wars children sixth final star wars ...     843  entertainment
4     berlin cheers antinazi film german movie antin...    1328  entertainment
...                                                 ...     ...            ...
1485  davies favours gloucester future wales hooker ...     715          sport
1486  beijingers fume parking fees choking traffic j...    1379       business
1487  cars pull us retail figures us retail sales fe...    1274       business
1488  rem announce new glasgow concert us band rem a...    1147  entertainment
1489  political squabbles snowball become commonplac...    2819       politics

[1490 rows x 3 columns]


In [23]:
print('total length of the data is:            {}'.format(new_df.shape[0]))
new_df['Category'].value_counts()

total length of the data is:            1490


Category
sport            346
business         336
politics         274
entertainment    273
tech             261
Name: count, dtype: int64

In [24]:
le = LabelEncoder()

le.fit(df['Category'])

df['target'] = le.transform(df['Category'])
df.head(10)

LabelEncoder()

,Text,Category,Categoryid,target
0,worldcom boss left books alone former worldc...,business,0,0
1,yeading face newcastle in fa cup premiership s...,sport,1,3
2,blair prepares to name poll date tony blair is...,politics,2,2
3,last star wars not for children the sixth an...,entertainment,3,1
4,berlin cheers for anti-nazi film a german movi...,entertainment,3,1
5,hague given up his pm ambition former conser...,politics,2,2
6,moya emotional after davis cup win carlos moya...,sport,1,3
7,s korean credit card firm rescued south korea ...,business,0,0
8,connors boost for british tennis former world ...,sport,1,3
9,japanese banking battle at an end japan s sumi...,business,0,0


In [25]:
from sklearn.model_selection import train_test_split

X = df['Text']
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,test_size=0.25,
                                                    random_state=500,shuffle=True)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from nltk import TweetTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score

tokenizer = TweetTokenizer(preserve_case=False)

In [27]:
linear_svc = Pipeline([('tfidf', TfidfVectorizer(lowercase=True, stop_words=stopwords_list,
                        tokenizer=tokenizer.tokenize)),('clf', LinearSVC(class_weight='balanced'))])

linear_svc.fit(X_train, y_train)

C:\Users\abhis\anaconda3\envs\tf1\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\abhis\anaconda3\envs\tf1\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x0000016DDEC112B0>>)),
                ('clf', LinearSVC(class_weight='balanced'))])

In [28]:
# prediction
y_hat_train=linear_svc.predict(X_train)
y_hat_test = linear_svc.predict(X_test)

from sklearn import metrics
import itertools  


svc_test_acc = round(accuracy_score(y_test, y_hat_test), 2)
svc_test_acc


0.98

In [29]:
# Assuming X_test[1] is the specific sentence you want to predict on
single_sentence = X_test[1]
y1_hat_test = linear_svc.predict([single_sentence])  # Wrap it in a list


In [33]:
df_combined = pd.concat([X_test, dfs_combined], axis=0)

In [34]:
print(y1_hat_test)

[3]


In [36]:
y3_hat_test = linear_svc.predict(dfs_combined)


In [37]:
print(y3_hat_test)

[0 4 0 2 2 2 0 0 2 4 2 2 2 2 2 2 2 2 1 2 2 0 0 4 4]
